In [3]:


from __future__ import annotations
import numpy as np
import pandas as pd
from typing import Tuple, Optional

#Global parameters
SEED = 42
AREA = 100.0                       # [m] square area [0,100]^2
UE_REGION = (30.0, 70.0)           # UE Monte-Carlo sampling box
STEPS = [20.0, 15.0]               # coarse grid steps to test (→ 36 and 49 pts)
M = 9                              # anchors used by WLS
n_pl = 2.0                         # path-loss exponent
sigma_db = 2.0                     # shadowing stdev [dB]
RSS0 = -30.0                       # reference RSS at 1 m [dBm] (arbitrary, consistent)
v = 3.0                            # horizontal speed [m/s]
t_h = 4.0                          # hover per coarse waypoint [s]
t_h_ref = 4.0                      # hover per refinement waypoint [s]

# NLoS model: single vertical wall
WALL_X = 60.0
L_pen_db = 9.0

# Warm-zone refinement
STEP_SEED = 10.0                   # coarse heatmap seed step [m]
STEP_REF = 5.0                     # refinement step [m]
WARM_SIDE = 25.0                   # warm zone side [m]

# Monte-Carlo runs
R = 100

# FAST path “enhancers”
K_AVG = 5
ALPHA_NLOS_UNCORR = 0.2
CV_P_CORRECT  = 0.90
CV_CONF_THRESH = 0.80

rng = np.random.default_rng(SEED)

def grid_points(step: float) -> np.ndarray:
    xs = np.arange(0.0, AREA + 1e-9, step)
    ys = np.arange(0.0, AREA + 1e-9, step)
    X, Y = np.meshgrid(xs, ys, indexing='xy')
    return np.column_stack([X.ravel(), Y.ravel()])

def is_blocked(a: np.ndarray, u: np.ndarray) -> bool:
    ax, ay = a; ux, uy = u
    return (ax - WALL_X) * (ux - WALL_X) < 0.0

def simulate_rss(anchors: np.ndarray, ue: np.ndarray, K: int = 1) -> Tuple[np.ndarray, np.ndarray]:
    rss_vec, L_vec = [], []
    for a in anchors:
        loss_pen = L_pen_db if is_blocked(a, ue) else 0.0
        snaps = []
        for _ in range(K):
            d = np.linalg.norm(ue - a)
            Xsig = rng.normal(0.0, sigma_db)
            snaps.append(RSS0 - 10*n_pl*np.log10(max(d, 1e-3)) + Xsig - loss_pen)
        rss_vec.append(float(np.mean(snaps)))
        L_vec.append(loss_pen)
    return np.array(rss_vec), np.array(L_vec)

def apply_cv_correction(rss_raw: np.ndarray, L_pen: np.ndarray) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    N = rss_raw.size
    is_block = (L_pen > 0.0)
    correct = rng.random(N) < CV_P_CORRECT
    conf    = rng.uniform(0.7, 1.0, size=N)
    use_corr = is_block & correct & (conf >= CV_CONF_THRESH)
    rss_corr = rss_raw + L_pen * use_corr.astype(float)
    return rss_corr, use_corr.astype(float), np.where(use_corr, conf, 0.0)

def invert_distance(rss: np.ndarray) -> np.ndarray:
    return 10.0 ** ((RSS0 - rss) / (10.0 * n_pl))

def select_M_nearest(anchors_all: np.ndarray, ue: np.ndarray, M: int) -> np.ndarray:
    d = np.linalg.norm(anchors_all - ue, axis=1)
    idx = np.argsort(d)[:M]
    return anchors_all[idx]

def select_M_visibility(anchors_all: np.ndarray, ue: np.ndarray, M: int) -> np.ndarray:
    """Prefer LoS anchors; fill with nearest NLoS if not enough."""
    d = np.linalg.norm(anchors_all - ue, axis=1)
    blocked_mask = np.array([is_blocked(a, ue) for a in anchors_all])
    idx_los  = np.argsort(d[~blocked_mask])
    idx_nlos = np.argsort(d[blocked_mask])
    los_ids  = np.where(~blocked_mask)[0][idx_los]
    nlos_ids = np.where(blocked_mask)[0][idx_nlos]
    pick = list(los_ids[:M])
    if len(pick) < M:
        pick += list(nlos_ids[:(M - len(pick))])
    return anchors_all[np.array(pick)]

def wls_position_weighted(anchors: np.ndarray, dists: np.ndarray, w: np.ndarray) -> np.ndarray:
    """Weighted linearized WLS (closest anchor as reference)."""
    ref = int(np.argmin(dists))
    x1, y1 = anchors[ref]; d1 = dists[ref]
    rows, rhs, w_rows = [], [], []
    for i in range(len(anchors)):
        if i == ref:
            continue
        xi, yi = anchors[i]; di = dists[i]
        rows.append([2*(xi - x1), 2*(yi - y1)])
        rhs.append(xi**2 + yi**2 - di**2 - (x1**2 + y1**2 - d1**2))
        w_rows.append(w[i])
    A = np.array(rows, float); b = np.array(rhs, float); wr = np.sqrt(np.array(w_rows, float))
    Aw = A * wr[:, None]; bw = b * wr
    sol, *_ = np.linalg.lstsq(Aw, bw, rcond=None)
    return sol

def heatmap_residual_min(anchors: np.ndarray, rss_meas: np.ndarray,
                         step: float, warm_center: Optional[np.ndarray]) -> np.ndarray:
    """Grid-search (x,y) minimizing squared residuals to measured RSS (raw model)."""
    if warm_center is None:
        xs = np.arange(0.0, AREA + 1e-9, step)
        ys = np.arange(0.0, AREA + 1e-9, step)
    else:
        cx, cy = warm_center
        half = WARM_SIDE / 2.0
        xs = np.arange(max(0.0, cx - half), min(AREA, cx + half) + 1e-9, step)
        ys = np.arange(max(0.0, cy - half), min(AREA, cy + half) + 1e-9, step)
    best = None; best_cost = np.inf
    for x in xs:
        for y in ys:
            p = np.array([x, y])
            pred = []
            for a in anchors:
                d = np.linalg.norm(p - a)
                loss_pen = L_pen_db if is_blocked(a, p) else 0.0
                pred.append(RSS0 - 10*n_pl*np.log10(max(d, 1e-3)) - loss_pen)
            pred = np.array(pred)
            cost = float(np.sum((rss_meas - pred)**2))
            if cost < best_cost:
                best_cost = cost
                best = p
    return best

def coarse_time(step: float, v: float=v, t_h: float=t_h):
    pts = grid_points(step)
    N_pts = pts.shape[0]
    n_x = int(AREA/step) + 1
    n_y = n_x
    D = AREA*(n_y-1) + step*(n_x-1)*n_y
    T = D/v + N_pts * t_h
    return N_pts, D, T

def refine_time(step_ref: float=STEP_REF, warm_side: float=WARM_SIDE, v: float=v, t_h_ref: float=t_h_ref):
    n = int(round(warm_side/step_ref)) + 1
    N_pts = n*n
    D = warm_side*(n-1) + step_ref*(n-1)*n
    T = D/v + N_pts * t_h_ref
    return N_pts, D, T

# One run under chosen flags
def run_trials_for_mode(R: int, step: float,
                        use_cv_fast: bool,
                        visibility_aware_m: bool,
                        weighted_wls: bool = True) -> pd.DataFrame:
    anchors_all = grid_points(step)
    rows = []
    for _ in range(R):
        ue = np.array([rng.uniform(*UE_REGION), rng.uniform(*UE_REGION)])

        # FAST: select anchors
        anchors_M = select_M_visibility(anchors_all, ue, M) if visibility_aware_m else select_M_nearest(anchors_all, ue, M)
        blocked_flags = np.array([is_blocked(a, ue) for a in anchors_M], dtype=bool)

        # measurements (+ temporal averaging)
        rss_raw, L_pen = simulate_rss(anchors_M, ue, K=K_AVG)

        # CV correction (oracle off; this mode compares realistic CV vs none)
        if use_cv_fast:
            rss_used, corrected_mask, conf_used = apply_cv_correction(rss_raw, L_pen)
        else:
            rss_used, corrected_mask, conf_used = rss_raw, np.zeros_like(rss_raw), np.zeros_like(rss_raw)

        d_used = invert_distance(rss_used)

        # row weights
        w = np.ones_like(rss_used)
        # down-weight uncorrected NLoS
        uncorrected = blocked_flags & (corrected_mask <= 0.0)
        w[uncorrected] = ALPHA_NLOS_UNCORR
        # mildly scale corrected rays by confidence
        corrected = blocked_flags & (corrected_mask > 0.0)
        w[corrected] = 0.8 + 0.2 * np.clip(conf_used[corrected], 0.0, 1.0)

        # solve
        p_fast = wls_position_weighted(anchors_M, d_used, w) if weighted_wls else wls_position_weighted(anchors_M, d_used, np.ones_like(w))

        # ROBUST: seed 10 m → refine 5 m
        rss_all, _ = simulate_rss(anchors_all, ue, K=K_AVG)
        p_seed = heatmap_residual_min(anchors_all, rss_all, step=STEP_SEED, warm_center=None)
        p_heat = heatmap_residual_min(anchors_all, rss_all, step=STEP_REF, warm_center=p_seed)

        err = lambda p: float(np.linalg.norm(p - ue))
        rows.append({
            "err_fast": err(p_fast),
            "err_robust": err(p_heat),
            "fast_blocked_frac": float(np.mean(blocked_flags)),
            "fast_corr_frac": float(np.mean(corrected_mask)),
            "fast_mean_w": float(np.mean(w)),
            "fast_los_count": int((~blocked_flags).sum()),
        })
    return pd.DataFrame(rows)

def summarize(df: pd.DataFrame, label: str, step: float) -> dict:
    N_pts, D, T = coarse_time(step)
    N_ref_pts, D_ref, T_ref = refine_time()
    return {
        "Mode": label,
        "Coarse step [m]": float(step),
        "N_pts (coarse)": int(N_pts),
        "FAST: RMSE [m]": float(df["err_fast"].mean()),
        "ROBUST: RMSE [m]": float(df["err_robust"].mean()),
        "FAST: blocked frac": float(df["fast_blocked_frac"].mean()),
        "FAST: corrected rays frac": float(df["fast_corr_frac"].mean()),
        "FAST: mean WLS weight": float(df["fast_mean_w"].mean()),
        "FAST: mean LoS count": float(df["fast_los_count"].mean()),
        "Coarse time T [min]": float(T/60.0),
        "Refine (per warm zone) time [min]": float(T_ref/60.0),
        "Refine N_pts": int(N_ref_pts),
    }

def run_all():
    all_rows = []
    for step in STEPS:
        dfA = run_trials_for_mode(R, step, use_cv_fast=True, visibility_aware_m=True, weighted_wls=True)
        all_rows.append(summarize(dfA, label="A: visibility-aware (CV on)", step=step))
        dfB = run_trials_for_mode(R, step, use_cv_fast=True, visibility_aware_m=False, weighted_wls=True)
        all_rows.append(summarize(dfB, label="B: nearest-anchors (CV on)", step=step))
    summary = pd.DataFrame(all_rows)
    for c in summary.columns:
        if summary[c].dtype.kind in "fc":
            summary[c] = summary[c].round(3)
    return summary

def save_artifacts(summary: pd.DataFrame):
    summary.to_csv("fast_vs_robust_summary.csv", index=False)
    cols = ["Mode","Coarse step [m]","N_pts (coarse)",
            "FAST: RMSE [m]","ROBUST: RMSE [m]",
            "FAST: blocked frac","FAST: corrected rays frac",
            "Coarse time T [min]","Refine (per warm zone) time [min]","Refine N_pts"]
    align = "lrrrrrrrrr"
    header = " & ".join(cols) + r" \\ \midrule" + "\n"
    body = "\n".join(" & ".join(map(str, row)) + r" \\" for row in summary[cols].values)
    latex = (
        r"\begin{table}[t]" "\n"
        r"  \centering" "\n"
        r"  \caption{Fast vs robust under two selection policies: visibility-aware vs nearest-anchors (both with CV).}" "\n"
        r"  \label{tab:fast-vs-robust-modes}" "\n"
        r"  \begin{tabular}{" + align + "}\n"
        r"    \toprule" "\n"
        f"    {header}"
        f"{body}\n"
        r"    \\" "\n"
        r"    \bottomrule" "\n"
        r"  \end{tabular}" "\n"
        r"\end{table}" "\n"
    )
    with open("fast_vs_robust_table.tex","w") as f:
        f.write(latex)

def main():
    summary = run_all()
    save_artifacts(summary)
    print("Saved: fast_vs_robust_summary.csv, fast_vs_robust_table.tex")
    print("\nSummary:\n", summary.to_string(index=False))

if __name__ == "__main__":
    main()


Saved: fast_vs_robust_summary.csv, fast_vs_robust_table.tex

Summary:
                        Mode  Coarse step [m]  N_pts (coarse)  FAST: RMSE [m]  ROBUST: RMSE [m]  FAST: blocked frac  FAST: corrected rays frac  FAST: mean WLS weight  FAST: mean LoS count  Coarse time T [min]  Refine (per warm zone) time [min]  Refine N_pts
A: visibility-aware (CV on)             20.0              36           2.170             1.969               0.000                      0.000                  1.000                  9.00                8.511                              3.928            36
 B: nearest-anchors (CV on)             20.0              36           5.309             2.253               0.146                      0.090                  0.954                  7.69                8.511                              3.928            36
A: visibility-aware (CV on)             15.0              49           1.763             2.058               0.000                      0.000                 